### Instruction 

- use github codesapce since it will setup all the tools for you, https://github.com/codespaces/new?hide_repo_select=true&ref=main&repo=599293758&machine=standardLinux32gb&devcontainer_path=.devcontainer%2Fdevcontainer.json&location=WestUs2

- az login --use-device-code
- az account set --subscription "e1b2a578-b466-409c-9466-cf5a7852d9ae"
- azd auth login
- azd up
- then give the parameter names , for openai select sweden centeral

### prepdocs.py
the **prepdocs.py** code : https://github.com/Azure-Samples/azure-search-openai-demo/blob/main/scripts/prepdocs.py is main for data processing . The script is designed to prepare documents by extracting content from PDFs, splitting the content into sections, uploading to Azure Blob Storage, and indexing in an Azure AI Search index. It uses various Azure services and custom libraries to accomplish these tasks. this code use all python libaray and classes in this folder https://github.com/Azure-Samples/azure-search-openai-demo/tree/main/scripts/prepdocslib . pdfparser.py decides whether to compute embeddings (text representations) using either Azure OpenAI services or  OpenAI services. If image searching is enabled, it retrieves the necessary Azure AI Vision key and sets up image embeddings.

 #### 1-pdfparser.py
- **prepdocs.py**  use **pdfparser.py** from prepdocslib folder .**pdfparser.py**  determines the appropriate PDF parser to use based on whether local or Azure services are specified. basicly this code  provides two ways to parse a PDF and extract its content: one for local parsing with PyPDF and another using Azure document inteligent for more advanced analysis including table extraction.The table_to_html class method: Converts a DocumentTable object from Azure AI Document Intelligence into an HTML table string. This method properly formats headers and spans for rows and columns, and escapes HTML characters in cell content.

#### 2-textsplitter.py
- **prepdocs.py**  use **textsplitter.py** from prepdocslib folder .**textsplitter.py** is more complex and does the actual work of splitting the text. It has various parameters that define sentence endings, word breaks, the maximum length of a section, a limit for searching for sentence endings, and an overlap size for the sections. It also has a verbose mode for logging and a flag to indicate whether image embeddings are being used.Special care is taken to handle tables within the text; if a section ends with an unclosed table, the splitting process adjusts the start of the next section to include the table, avoiding splitting tables across sections.
In summary, this code is designed to split the text content of documents into manageable sections that can be processed by embedding models, while maintaining the integrity of sentences and tables

#### 3-strategy.py
- **prepdocs.py**  use **strategy.py** from prepdocslib folder .**strategy.py** have two classes: SearchInfo and Strategy.SearchInfo class:This class represents a connection to an Azure Search service. Strategy class:defines a strategy for ingesting documents into the Azure Search service

#### 4-filestrategy.py
- **prepdocs.py**  use **filestrategy.py** from prepdocslib folder .**filestrategy.py** is Strategy for ingesting documents into a search service from files stored either locally or in a data lake storage account. It initializes with multiple components:
   - list_file_strategy: For listing files to be processed.
   - blob_manager: For managing blobs (binary large objects) like uploading and removing.
   - pdf_parser: For parsing PDF documents.
   - text_splitter: For splitting text into sections or pages.
   - document_action: To specify the action to be taken (Add, Remove, or RemoveAll).
   - embeddings: For handling textual embeddings, optional and specific to OpenAI embeddings.
   - image_embeddings: For handling image embeddings, also optional.
   - search_analyzer_name, use_acls, and category: Additional optional parameters for configuring the search.
- **filestrategy.py** use https://github.com/Azure-Samples/azure-search-openai-demo/blob/main/scripts/prepdocslib/searchmanager.py too
   - **searchmanager.py** This code is a Python module that interacts with the Azure Search service to manage the indexing and searching of documents. <u>so all index configuration loactaed here.</u>
      -  The Section class represents a section of a page that will be stored in the search service, used for context by the Azure OpenAI service.
      - The SearchManager class is responsible for managing the search service. It can create search indexes and update or remove sections stored in these indexes. The class uses SearchInfo for settings and can optionally work with text and image embeddings provided by OpenAIEmbeddings.
      - The create_index async method creates a search index with defined fields. These fields include simple fields like id, category, sourcepage, and sourcefile, as well as complex fields for embeddings like embedding and imageEmbedding if images are included.
      - The search index is equipped with advanced configurations like semantic settings and vector search, using the HNSW (Hierarchical Navigable Small World) algorithm for efficient similarity search in high-dimensional spaces.https://learn.microsoft.com/en-us/azure/search/vector-search-ranking
      - The update_content async method is used to upload documents to the search index in batches. It handles the creation of document embeddings and the association of image embeddings if provided.
      - The remove_content async method removes documents from the search index. If a specific path is provided, it removes only documents related to that path; otherwise, it removes all documents.
      - The code is designed to handle large-scale operations by breaking down the content into batches and using asynchronous operations for efficient processing.
      - The module is structured to work with Azure's search capabilities, leveraging AI-enhanced indexing and searching for documents, potentially including features like semantic understanding and image content search.